In [1]:
import pandas as pd
import numpy as np

# This is the third go around for groupby

Though I'm pretty good with DataFrames now, I do still go to the documentation quite often. I'd like to make one more pass over the groupby section and then probably go look at the braodcasting. On with it.

## ex.0a Create a `DataFrame` from dictionary

In [2]:
df = pd.DataFrame({
    'a': np.random.randn(5),
    'b': np.random.randn(5),
    'c': np.random.randn(5)
})

In [3]:
df

,a,b,c
0,-1.174473,0.247760,0.502319
1,-0.162032,-0.528797,-1.206141
2,-0.010633,-1.527509,-0.211028
3,-0.906930,0.590233,-0.488118
4,0.726582,-0.396174,-1.736669


## ex.0b Select with a boolean array

In [4]:
mask = df['b'].values > 0

In [5]:
mask

array([ True, False, False,  True, False], dtype=bool)

In [6]:
g = df.groupby(mask)
g.size()

False    3
True     2
dtype: int64

## ex.1 Use `groupby` with a single key

In [7]:
df = pd.DataFrame({
    'key1': 'a a b b a'.split(),
    'key2': 'one two one two one'.split(),
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)
})

In [8]:
df

,data1,data2,key1,key2
0,-0.210657,-1.432751,a,one
1,-0.729698,1.614340,a,two
2,-0.247788,0.310028,b,one
3,0.196606,0.982720,b,two
4,0.072091,0.533157,a,one


In [9]:
g = df['data1'].groupby(df['key1'])

In [10]:
g.mean()

key1
a   -0.289421
b   -0.025591
Name: data1, dtype: float64

## ex.2 Use `groupby` with arrays not in the `DataFrame`

In [11]:
states = np.array('Ohio California California Ohio Ohio'.split())
years = np.array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.729698
            2006   -0.247788
Ohio        2005   -0.007026
            2006    0.072091
Name: data1, dtype: float64

## ex.3 Iterate over `grouped` object

In [13]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -0.210657 -1.432751    a  one
1 -0.729698  1.614340    a  two
4  0.072091  0.533157    a  one
b
      data1     data2 key1 key2
2 -0.247788  0.310028    b  one
3  0.196606  0.982720    b  two


In [14]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -0.210657 -1.432751    a  one
4  0.072091  0.533157    a  one
('a', 'two')
      data1    data2 key1 key2
1 -0.729698  1.61434    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.247788  0.310028    b  one
('b', 'two')
      data1    data2 key1 key2
3  0.196606  0.98272    b  two


## ex.4 Groub by columns based on `dtype`

In [15]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [16]:
g = df.groupby(df.dtypes, axis=1)

In [17]:
for dtype, group in g:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.210657 -1.432751
1 -0.729698  1.614340
2 -0.247788  0.310028
3  0.196606  0.982720
4  0.072091  0.533157
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## ex.5 Return `DataFrame` or `Series` from `groupby`

In [18]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.449797
     two   1.614340
b    one   0.310028
     two   0.982720

In [19]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one    -0.449797
      two     1.614340
b     one     0.310028
      two     0.982720
Name: data2, dtype: float64

Well that's interesting. I wonder why that happens.

In [20]:
# DataFrame
df.groupby(['key1', 'key2'])[['data2']]

In [21]:
# Series
df.groupby(['key1', 'key2'])['data2']

## ex.6a Use `iloc` for selecting cells

In [22]:
people = pd.DataFrame(np.random.randn(5, 5),
                     columns = 'a b c d e'.split(),
                     index='Joe Steve Wes Jim Travis'.split())
people.iloc[2:3, [1, 3]] = np.nan
people

,a,b,c,d,e
Joe,0.284988,-3.102663,-0.655100,-1.297323,1.876667
Steve,-0.545959,0.034753,-0.030276,1.505649,1.622833
Wes,-1.063619,NaN,-0.052710,NaN,2.404832
Jim,1.107038,3.259952,0.697896,0.380116,-0.624901
Travis,-0.222191,-0.082654,0.629598,-0.517107,0.323193


## ex.6b Same thing

In [23]:
df = pd.DataFrame()

for i in range(10):
    df[i] = np.arange(10)

df.iloc[[4, 8], 2:7]

,2,3,4,5,6
4,4,4,4,4,4
8,8,8,8,8,8


## ex.7a Dict comprehensions to create a mapping

In [24]:
mapping = {i:j for i, j in zip(
    'a b c d e f'.split(),
    'red red blue blue red orange'.split())
  }
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

## ex.7b Use a mapping to group a `DataFrame`

In [25]:
people

,a,b,c,d,e
Joe,0.284988,-3.102663,-0.655100,-1.297323,1.876667
Steve,-0.545959,0.034753,-0.030276,1.505649,1.622833
Wes,-1.063619,NaN,-0.052710,NaN,2.404832
Jim,1.107038,3.259952,0.697896,0.380116,-0.624901
Travis,-0.222191,-0.082654,0.629598,-0.517107,0.323193


In [26]:
by_col = people.groupby(mapping, axis=1)
by_col.sum()

,blue,red
Joe,-1.952423,-0.941008
Steve,1.475373,1.111627
Wes,-0.052710,1.341214
Jim,1.078012,3.742088
Travis,0.112491,0.018348


## ex.7c New mapping

In [27]:
new_map = {i:j for i, j in zip(
    people.columns,
    'fee fi foe fi fum'.split()
)}
g = people.groupby(new_map, axis=1)
g.sum()

,fee,fi,foe,fum
Joe,0.284988,-4.399986,-0.655100,1.876667
Steve,-0.545959,1.540402,-0.030276,1.622833
Wes,-1.063619,NaN,-0.052710,2.404832
Jim,1.107038,3.640068,0.697896,-0.624901
Travis,-0.222191,-0.599761,0.629598,0.323193


## ex.8a Convert `dict` to `Series`

In [28]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

## ex.8b Use `Series` to group a `DataFrame`

In [29]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## ex.9 Use `groupby` with a function

ie. the function takes as an argument, the index value of the record.

In [30]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.328407,0.157289,-0.009914,-0.917207,3.656599
5,-0.545959,0.034753,-0.030276,1.505649,1.622833
6,-0.222191,-0.082654,0.629598,-0.517107,0.323193


Any function passed as a group key will be called once per index value, with the return values being used as the group names.

## ex.10 Use `groupby` with mixed grouping types

In [31]:
key_list = 'one one one two two'.split()
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.063619 -3.102663 -0.655100 -1.297323  1.876667
  two  1.107038  3.259952  0.697896  0.380116 -0.624901
5 one -0.545959  0.034753 -0.030276  1.505649  1.622833
6 two -0.222191 -0.082654  0.629598 -0.517107  0.323193

## ex.11a Create a hierarchical index

In [32]:
columns = pd.MultiIndex.from_arrays([
    'US US US JP JP'.split(),
    [1, 3, 5, 1, 3]],
    names=['city', 'tenor'])

In [33]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.417383 -0.338375  0.387282 -0.498013  1.328769
1      0.867349  0.635869  0.292570  0.387674  0.082473
2      3.265680  0.456849 -0.026657  0.960542 -0.243650
3     -0.070533  0.246211  0.784102 -1.259141  0.028232

## ex. 11b Use `groupby` with hierarchical index

In [34]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## ex.12 Get a group's `quantile`

The key here is that `quantile()` is a method of the `Series` object, not the `grouped` object. However, it is available as the result of a `groupby` operation.

In [35]:
df = pd.DataFrame({
    'key1': 'a a b b a'.split(),
    'key2': 'one two one two one'.split(),
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)
})

In [36]:
df

,data1,data2,key1,key2
0,0.539133,0.942189,a,one
1,-0.514010,2.131087,a,two
2,-1.748183,-0.559265,b,one
3,-2.584400,-0.949097,b,two
4,2.240124,-1.613481,a,one


In [37]:
g = df.groupby('key1')
g['data1'].quantile(0.9)

key1
a    1.899926
b   -1.831805
Name: data1, dtype: float64

## ex.13 Create your own aggregation function

In [38]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [39]:
g.agg(peak_to_peak)

,data1,data2
key1,,
a,2.754134,3.744568
b,0.836217,0.389831


## ex.14 Create a new column from existing columns

In [40]:
tips = pd.read_csv('../data/tips.csv')

In [41]:
tips.head(1)

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2


In [42]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head(1)

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447


## ex.15 Aggregate with multiple functions at a time

In [43]:
g = tips.groupby(['day', 'smoker'])
g_pct = g['tip_pct']
g_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [44]:
g_pct.agg(['mean', 'std', peak_to_peak], as_index=False)

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

## ex.16 The `apply` method

In [45]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [46]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## ex.17 `apply` can be used on `groupby` objects

In [47]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

## ex.18 Keyword arguments in `apply`

In [48]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

## ex.19 Suppress the group keys

In [49]:
foo = tips.groupby('smoker', group_keys=False).apply(top)
foo

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## ex.20 Off topic: the `pd.cut()` function

This is not a `groupby` thing and requires it's own notebook. More on this another time.

In [50]:
# bins = [0, 5, 10, 15, 20, 999]  # You can specify your own buckets
# bar = pd.cut(foo['total_bill'], bins)

bar = pd.cut(foo['total_bill'], 3, precision=2)  # Or you can let pandas pick them

In [51]:
bar[:3]

88     (17.5, 24.71]
185    (17.5, 24.71]
51     (10.28, 17.5]
Name: total_bill, dtype: category
Categories (3, interval[float64]): [(3.05, 10.28] < (10.28, 17.5] < (17.5, 24.71]]

In [52]:
bar.values.codes

array([2, 2, 1, 0, 1, 1, 2, 0, 0, 0], dtype=int8)

In [53]:
pd.value_counts(bar)

(3.05, 10.28]    4
(17.5, 24.71]    3
(10.28, 17.5]    3
Name: total_bill, dtype: int64

# Part Two: Recipes

## ex.21 Quantile bucket analysis

In [54]:
frame = pd.DataFrame({
    'data1': np.random.randn(1000),
    'data2': np.random.randn(1000)
})

In [55]:
quartiles = pd.cut(frame['data1'], 4)                 # Equal length buckets
quantiles = pd.qcut(frame['data1'], 4, labels=False)  # Equal size buckets

In [56]:
quartiles[:3]

0     (0.167, 1.765]
1    (-1.431, 0.167]
2     (0.167, 1.765]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.035, -1.431] < (-1.431, 0.167] < (0.167, 1.765] < (1.765, 3.363]]

In [57]:
def get_stats(group):
    return {
        'min': group.min(),
        'max': group.max(),
        'count': group.count(),
        'mean': group.mean()
    }

In [58]:
gl = frame['data2'].groupby(quartiles)  # Equal length buckets
gs = frame['data2'].groupby(quantiles)  # Equal size buckets

In [59]:
gl.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.035, -1.431]",60.0,2.794118,0.025248,-1.462207
"(-1.431, 0.167]",480.0,2.972815,0.000387,-2.780031
"(0.167, 1.765]",422.0,3.489362,0.044010,-2.922424
"(1.765, 3.363]",38.0,2.826373,-0.078849,-2.864282


In [60]:
gs.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,250.0,2.960364,-0.070797,-2.780031
1,250.0,2.972815,0.070280,-2.636054
2,250.0,3.489362,0.041582,-2.394045
3,250.0,2.977965,0.028043,-2.922424


## ex.22 Filling in missing values with group specific values

In [61]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4

data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.326154
New York     -0.153364
Vermont            NaN
Florida       1.227871
Oregon       -0.658547
Nevada             NaN
California   -0.454738
Idaho              NaN
dtype: float64

In [62]:
data.groupby(group_key).mean()

East   -0.083882
West   -0.556642
dtype: float64

Fill values based on group data

In [63]:
fill_mean = lambda g: g.fillna(g.mean())

data.groupby(group_key).apply(fill_mean)

Ohio         -1.326154
New York     -0.153364
Vermont      -0.083882
Florida       1.227871
Oregon       -0.658547
Nevada       -0.556642
California   -0.454738
Idaho        -0.556642
dtype: float64

Fill with predefined values

In [64]:
fill_values = {'East':0.5, 'West':-1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio         -1.326154
New York     -0.153364
Vermont       0.500000
Florida       1.227871
Oregon       -0.658547
Nevada       -1.000000
California   -0.454738
Idaho        -1.000000
dtype: float64

## ex.23 Random sampling and permutation

In [65]:
suits = 'H S C D'.split()
card_val = (list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']
cards = []

In [66]:
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [67]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [68]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

8C      8
KH     10
10D    10
JC     10
JD     10
dtype: int64

Sample one card from each suit

In [69]:
get_suit = lambda card: card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  KC     10
   10C    10
D  QD     10
   5D      5
H  9H      9
   5H      5
S  7S      7
   10S    10
dtype: int64

In [70]:
# Also this
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

QC    10
6C     6
3D     3
6D     6
5H     5
9H     9
5S     5
AS     1
dtype: int64